# Data_Loader
## 我们将使用 TensorFlow 2 构建数据导入Pipeline
#### 1、分类任务
#### 2、分割任务
#### 3、随机匹配

### 分类任务
1、 文件夹的目录形式如下

Father_Dir
- Apple
    - 1.jpg
    - 2.jpg
- Origin
    - 1.jpg
    - 2.jpg

In [1]:
import tensorflow as tf
import numpy as np
Father_Dir = r'C:\Users\liuye\Desktop\Father_Dir/'
# shuffle 默认为Ture
files_path_Dataset = tf.data.Dataset.list_files(Father_Dir + r'*/*.jpg', shuffle=False)
print(files_path_Dataset)
print('(正确)将shuffle设置为False，遍历迭代器：')
for i in files_path_Dataset.as_numpy_iterator():
    print(i)
print('(错误)将shuffle设置为False，遍历迭代器：')
for i in range(4):
    print(files_path_Dataset.as_numpy_iterator().next())

InvalidArgumentError: Expected 'tf.Tensor(False, shape=(), dtype=bool)' to be true. Summarized data: b'No files matched pattern: C:\\Users\\liuye\\Desktop\\Father_Dir/*/*.jpg'

现在我们能够得到不断输出jpg图像的路径的迭代器files_path

接下来我们可以编写load_function函数从文件名导入图像文件,transform函数预处理图像

In [ ]:
# 先构建image与label的配对
image_label_Dict = {"Apple": 0, "Orange": 1}

def transform(image):
    image = image / 255
    return image

def load_function(img_path, target_size=(32, 32), Transform=None, Dict=None):
    # 从路径导入图像，这一步可以使用opncv, tensorflow, PIL等库，可以根据自己要进行的图像处理库进行选择
    if Transform is None:
        Transform = transform
    if Dict is None:
        Dict = image_label_Dict
    image = tf.io.read_file(img_path)
    image = tf.image.decode_jpeg(image)
    if target_size != (0, 0):
        image = tf.image.resize(image, target_size)
    if Transform:
        image = Transform(image)
    # 如果是有监督，对应的image还需要Label
    # 在此案例中，Image分为了Apple与Orange,所以我们可以构建一个字典来进行标注
    label = tf.constant(2, dtype=tf.int8)
    if Dict:
        for key in Dict.keys():
            if tf.strings.regex_full_match(img_path, ".*{}.*".format(key)):
                label = tf.constant(Dict.get(key), dtype=tf.int8)

    return image, label

In [ ]:
data_train = files_path_Dataset.map(load_function)
for i, (images, labels) in enumerate(data_train.take(4)):
    print(images.shape)
    print(labels.numpy())

### 分割任务
1、 文件夹的目录形式如下

Father_Dir

- Image
    - 1.jpg
    - 2.jpg
- Segmentation
    - 1.png
    - 2.png

此时1.jpg对应1.png，2.jpg对应2.png
此时就不能使用`tf.data.Dataset.list_files`，因为一方面Image与Label是分离的，另一方面，Image与Label不是无序的，是存在相互对应的关系，
所以此时应该采取另外一种方式，即借助txt文件，txt的一行有两个参数分别对应Image与Label，中间用 `,` 隔开，到时候直接对txt文件进行处理就能构建好
pipeline。

**但是在示例中，Segmentation的文件格式jpg**

In [ ]:
file_txt = r'L:\ALASegmentationNets_v2\Data\Stage_4\train.txt'
train_img_file = r'L:\ALASegmentationNets_v2\Data\Stage_4\train\img/'
train_label_file = r'L:\ALASegmentationNets_v2\Data\Stage_4\train\mask/'
files_txt = np.loadtxt(file_txt, delimiter=',', dtype=bytes, encoding='utf-8')
path_ds = tf.data.Dataset.from_tensor_slices(files_txt)

print(path_ds.as_numpy_iterator().next())

In [ ]:
# 此时需要对load_function进行重写才能满足条件
# 对Transform要认真构造

transform_image = transform
transform_label = transform

def load_function(img_label_path, target_size=(448, 448),
                  Transform_image = transform_image,
                  Transform_Label = transform_label
                  ):
    # 从路径导入图像，这一步可以使用opncv, tensorflow, PIL等库，可以根据自己要进行的图像处理库进行选择
    # 注意图像文件格式！
    img_path, label_path = img_label_path[0], img_label_path[1]

    image = tf.io.read_file(train_img_file + img_path)
    image = tf.image.decode_jpeg(image)

    label = tf.io.read_file(train_label_file + label_path)
    label = tf.image.decode_jpeg(label)

    if target_size != (0, 0):
        image = tf.image.resize(image, target_size)
    if Transform_image:
        image = Transform_image(image)
    if Transform_Label:
        label = Transform_Label(label)

    return image, label

In [ ]:
data_train = path_ds.map(load_function)
for i, (images, labels) in enumerate(data_train.take(4)):
    print(images.shape)
    print(labels.shape)

### Cycle_GAN 非对称数据集
3、 文件夹的目录形式如下

Father_Dir
- Apple
    - 1.jpg
    - 2.jpg
- Origin
    - 1.jpg
    - 2.jpg

假如我们想训练一个神经网络能够把苹果变成橘子，也能把橘子变成苹果，此时虽然说Apple与Orange是对应的，但是在各子类中图像是可以无序的，
此时应该如何构建Pipeline呢？

其实非常简单，把Apple看成Image， Origin看出Label，将内部状态设定为无序就可以

In [ ]:
Father_Dir = r'C:\Users\liuye\Desktop\Father_Dir/'
# shuffle 默认为Ture
# 所以每次运行，其对应的文件都不一样，但类是相对应的
Apple_files_path_Dataset = tf.data.Dataset.list_files(Father_Dir + r'Apple/*.jpg', shuffle=True)
Orange_files_path_Dataset = tf.data.Dataset.list_files(Father_Dir + r'Orange/*.jpg', shuffle=True)
for i in range(3):
    for j in tf.data.Dataset.zip((Apple_files_path_Dataset, Orange_files_path_Dataset)):
        print(j)

此时我们设置好相应transform函数就能完成图像的载入了
请注意与第二部分的load_function的差别

In [ ]:
def load_function(img_path, target_size=(448, 448),
                  Transform_image = transform_image,
                  ):
    # 从路径导入图像，这一步可以使用opncv, tensorflow, PIL等库，可以根据自己要进行的图像处理库进行选择
    # 注意图像文件格式！

    image = tf.io.read_file(img_path)
    image = tf.image.decode_jpeg(image)

    if target_size != (0, 0):
        image = tf.image.resize(image, target_size)
    if Transform_image:
        image = Transform_image(image)

    return image

In [ ]:
# 注意这使用transform函数直接就是第二类的load_function
# 我们来看看效果把

data_train_Apple = Apple_files_path_Dataset.map(load_function)
data_train_Orange = Orange_files_path_Dataset.map(load_function)
data_train = tf.data.Dataset.zip((data_train_Apple, data_train_Orange))
for i, (apple, orange) in enumerate(data_train.take(4)):
    print(apple.shape)
    print(orange.shape)